# Euler Rodrigues Parameters Functions

In [1]:
import numpy as np

import sys
from pathlib import Path

# Dynamically add DCM_utils directory to path
sys.path.insert(0, str(Path('..').resolve()))
from DCM_utils import *

# 1) EP_to_DCM

In [2]:
def EP_to_DCM(q, convention="scalar_first"):
    """
    Converts the EP/Quaternion to a direction cosine matrix (C).

     Args:
        q (np.array): A numpy array of size 4 (a row vector) representing the quaternion.
                      Depending on the convention:
                        - "scalar_first": [q0, q1, q2, q3], where q0 is the scalar part.
                        - "scalar_last": [q1, q2, q3, q0], where q0 is the scalar part.

        convention (str): Specifies the convention for quaternion representation.
                          Options: "scalar_first" (default) or "scalar_last".

    Returns:
        np.array: A 3x3 rotation matrix (C).
    """
    # Validate input vector
    validate_vec4(q)
    
    # Ensure q is a float array to maintain precision
    q = np.array(q, dtype=np.float64)
    
    # Check that the holonomic constraint of quaternion is satisfied, else normalize it
    q_norm = np.linalg.norm(q)
    if not np.isclose(q_norm, 1.0, atol=1e-8):
        q /= q_norm
    
    # Adjust indexing based on the specified convention
    if convention == "scalar_last":
        q1, q2, q3, q0 = q  # Swap positions to treat q0 as the last element
    elif convention == "scalar_first":
        q0, q1, q2, q3 = q  # Default behavior
    else:
        raise ValueError(f"Invalid convention '{convention}'. Choose 'scalar_first' or 'scalar_last'.")
    print(f"q0: {q0}")
    print(f"q1: {q1}")
    print(f"q2: {q2}")
    print(f"q3: {q3}")
    # Compute the elements of the C
    C = np.array([
        [q0**2 + q1**2 - q2**2 - q3**2, 2 * (q1*q2 + q0*q3)          , 2 * (q1*q3 - q0*q2)          ],
        [2 * (q1*q2 - q0*q3)          , q0**2 - q1**2 + q2**2 - q3**2, 2 * (q2*q3 + q0*q1)          ],
        [2 * (q1*q3 + q0*q2)          , 2 * (q2*q3 - q0*q1)          , q0**2 - q1**2 - q2**2 + q3**2]
    ])
    
    return C

In [3]:
Q1 = [-0.191253573924084, 0.0703589614829628, -0.0855928926215986, 0.97526690897055]

EP_to_DCM(Q1, convention="scalar_last")

q0: 0.97526690897055
q1: -0.191253573924084
q2: 0.0703589614829628
q3: -0.0855928926215986


array([[ 0.97544695, -0.19386464, -0.10449764],
       [ 0.14003903,  0.91219185, -0.38509102],
       [ 0.16997743,  0.36100211,  0.91694337]])

## 1.1 - Functional testing of EP_to_DCM

In [4]:
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the EP2C function from the AVS lab
from RigidBodyKinematics import EP2C

# Define test quaternion vectors (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],         # Identity quaternion (no rotation)
    [0, 1, 0, 0],         # 180-degree rotation about x-axis
    [0, 0, 1, 0],         # 180-degree rotation about y-axis
    [0, 0, 0, 1],         # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],  # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],  # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],  # 90-degree rotation about z-axis
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)

    print(f"Test Case {i + 1}:")
    print(f"Quaternion (q) = {q}")

    # Compute DCM using the existing EP2C function
    C_existing = EP2C(q)

    # Compute DCM using your EP_to_DCM function
    C_custom = EP_to_DCM(q)

    # Ensure both are NumPy arrays for easy comparison
    C_existing = np.array(C_existing)
    C_custom = np.array(C_custom)

    # Calculate the difference between the two C matrices
    difference = C_existing - C_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between EP_to_DCM and EP2C: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("C matrices differ significantly.\n")
    else:
        print("C matrices match.\n")

    print("-" * 50)


Test Case 1:
Quaternion (q) = [1. 0. 0. 0.]
q0: 1.0
q1: 0.0
q2: 0.0
q3: 0.0
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 2:
Quaternion (q) = [0. 1. 0. 0.]
q0: 0.0
q1: 1.0
q2: 0.0
q3: 0.0
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 3:
Quaternion (q) = [0. 0. 1. 0.]
q0: 0.0
q1: 0.0
q2: 1.0
q3: 0.0
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 4:
Quaternion (q) = [0. 0. 0. 1.]
q0: 0.0
q1: 0.0
q2: 0.0
q3: 1.0
Max difference between EP_to_DCM and EP2C: 0.000000000000e+00
C matrices match.

--------------------------------------------------
Test Case 5:
Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
q0: 0.7071067811865476
q1: 0.7071067811865476
q2: 0.0
q3: 0.0
Max difference between EP_to_D

# 2) DCM_to_EP

In [5]:
def DCM_to_EP(C, convention="scalar_first"):
    """
    Converts a Direction Cosine Matrix (C) to a quaternion using Shepperd's method to ensure robustness against numerical issues.
    
    Args:
        C (np.array): A 3x3 rotation matrix (C).
        convention (str): Specifies the convention for quaternion representation.
                          Options: "scalar_first" (default) or "scalar_last".
    
    Returns:
        np.array: A quaternion represented as a numpy array of size 4.
                  Format depends on the `convention` parameter.
    """
    # Validate Input DCM
    validate_DCM(C)

    trace = np.trace(C)
    q_squared = np.zeros(4)
    q_squared[0] = (1.0 + trace) / 4.0
    q_squared[1] = (1.0 + 2 * C[0, 0] - trace) / 4.0
    q_squared[2] = (1.0 + 2 * C[1, 1] - trace) / 4.0
    q_squared[3] = (1.0 + 2 * C[2, 2] - trace) / 4.0

    q = np.zeros(4)
    max_index = np.argmax(q_squared)

    if max_index == 0:
        q[0] = np.sqrt(q_squared[0])
        q[1] = (C[1, 2] - C[2, 1]) / (4 * q[0])
        q[2] = (C[2, 0] - C[0, 2]) / (4 * q[0])
        q[3] = (C[0, 1] - C[1, 0]) / (4 * q[0])
    
    elif max_index == 1:
        q[1] = np.sqrt(q_squared[1])
        q[0] = (C[1, 2] - C[2, 1]) / (4 * q[1])
        if q[0] < 0:
            q[0] = -q[0]
            q[1] = -q[1]
        q[2] = (C[0, 1] + C[1, 0]) / (4 * q[1])
        q[3] = (C[2, 0] + C[0, 2]) / (4 * q[1])
        
    elif max_index == 2:
        q[2] = np.sqrt(q_squared[2])
        q[0] = (C[2, 0] - C[0, 2]) / (4 * q[2])
        if q[0] < 0:
            q[0] = -q[0]
            q[2] = -q[2]
        q[1] = (C[0, 1] + C[1, 0]) / (4 * q[2])
        q[3] = (C[1, 2] + C[2, 1]) / (4 * q[2])

    elif max_index == 3:
        q[3] = np.sqrt(q_squared[3])
        q[0] = (C[0, 1] - C[1, 0]) / (4 * q[3])
        if q[0] < 0:
            q[0] = -q[0]
            q[3] = -q[3]
        q[1] = (C[2, 0] + C[0, 2]) / (4 * q[3])
        q[2] = (C[1, 2] + C[2, 1]) / (4 * q[3])

    # Adjust output based on the specified convention
    if convention == "scalar_last":
        q = np.array([q[1], q[2], q[3], q[0]])
    elif convention == "scalar_first":
        q = np.array([q[0], q[1], q[2], q[3]])
    else:
        raise ValueError(f"Invalid convention '{convention}'. Choose 'scalar_first' or 'scalar_last'.")

    return q

## 2.1 - Functional testing of DCM_to_EP

In [6]:
# Define test quaternions (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],                                                  # Identity quaternion (no rotation)
    [0, 1, 0, 0],                                                  # 180-degree rotation about x-axis
    [0, 0, 1, 0],                                                  # 180-degree rotation about y-axis
    [0, 0, 0, 1],                                                  # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],                            # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],                            # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],                            # 90-degree rotation about z-axis
    [np.sqrt(0.25), np.sqrt(0.25), np.sqrt(0.25), np.sqrt(0.25)],  # General rotation
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Input Quaternion (q) = {q}")

    # Normalize quaternion to ensure it represents a valid rotation
    q_normalized = q / np.linalg.norm(q)

    # Compute DCM from quaternion using EP_to_DCM function
    C = EP_to_DCM(q_normalized)

    # Get quaternion back from DCM using DCM_to_EP function
    q_reconstructed = DCM_to_EP(C)

    # Normalize reconstructed quaternion to ensure valid comparison
    q_reconstructed_normalized = q_reconstructed / np.linalg.norm(q_reconstructed)

    # Calculate the difference between the original and reconstructed quaternions
    difference = q_normalized - q_reconstructed_normalized
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Reconstructed Quaternion (q) = {q_reconstructed_normalized}")
    print(f"Max difference: {max_diff:.12e}")

    if max_diff > 1e-12:
        print("Quaternions differ significantly.\n")
    else:
        print("Quaternions match.\n")

    print("-" * 50)


Test Case 1:
Input Quaternion (q) = [1. 0. 0. 0.]
q0: 1.0
q1: 0.0
q2: 0.0
q3: 0.0
Reconstructed Quaternion (q) = [1. 0. 0. 0.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 2:
Input Quaternion (q) = [0. 1. 0. 0.]
q0: 0.0
q1: 1.0
q2: 0.0
q3: 0.0
Reconstructed Quaternion (q) = [0. 1. 0. 0.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 3:
Input Quaternion (q) = [0. 0. 1. 0.]
q0: 0.0
q1: 0.0
q2: 1.0
q3: 0.0
Reconstructed Quaternion (q) = [0. 0. 1. 0.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 4:
Input Quaternion (q) = [0. 0. 0. 1.]
q0: 0.0
q1: 0.0
q2: 0.0
q3: 1.0
Reconstructed Quaternion (q) = [0. 0. 0. 1.]
Max difference: 0.000000000000e+00
Quaternions match.

--------------------------------------------------
Test Case 5:
Input Quaternion (q) = [0.70710678 0.70710678 0.       

# 3) Bmat_EP

In [7]:
def Bmat_EP(q, convention="scalar_first"):
    """
    Computes the 4x3 B matrix that maps body angular velocity (omega) to the derivative of the quaternion (Euler parameters) vector.

        dQ/dt = 1/2 * [B(Q)] * omega

    Args:
        q (array-like): A 4-element quaternion (Euler parameter) vector [q0, q1, q2, q3].
        convention (str): Specifies the convention for quaternion representation.
                          Options ---> "scalar_first" (default) or "scalar_last"
    
    Returns:
        np.ndarray: A 4x3 B matrix.
    
    Notes:
        - The quaternion vector q should be in the form [q0, q1, q2, q3], where q0 is the scalar component.
    """
    # Validate the input quaternion vector
    validate_vec4(q)

    # Convert input to a NumPy array if not already
    q = np.array(q, dtype=float)

    # Extract components of the quaternion
    if convention == "scalar_first":
        q0, q1, q2, q3 = q
    elif convention == "scalar_last":
        q1, q2, q3, q0 = q

    # Construct the B matrix using a structured array
    B = np.array([[-q1, -q2, -q3],
                  [ q0, -q3,  q2],
                  [ q3,  q0, -q1],
                  [-q2,  q1,  q0]])

    return B

Bmat_EP([0,1,2,1])

array([[-1., -2., -1.],
       [ 0., -1.,  2.],
       [ 1.,  0., -1.],
       [-2.,  1.,  0.]])

## 3.1 - Functional testing of Bmat_EP

In [8]:
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the BmatEP function from the AVS lab
from RigidBodyKinematics import BmatEP

# Define test quaternion vectors (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],                        # Identity quaternion (no rotation)
    [0, 1, 0, 0],                        # 180-degree rotation about x-axis
    [0, 0, 1, 0],                        # 180-degree rotation about y-axis
    [0, 0, 0, 1],                        # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],  # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],  # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],  # 90-degree rotation about z-axis
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Quaternion (q) = {q}")

    # Compute B matrix using the existing BmatEP function
    B_existing = BmatEP(q)

    # Compute B matrix using your Bmat_EP function
    B_custom = Bmat_EP(q)

    # Ensure both are NumPy arrays for easy comparison
    B_existing = np.array(B_existing)
    B_custom = np.array(B_custom)

    # Calculate the difference between the two B matrices
    difference = B_existing - B_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between Bmat_EP and BmatEP: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("B matrices differ significantly.\n")
    else:
        print("B matrices match.\n")

    print("-" * 50)


Test Case 1:
Quaternion (q) = [1. 0. 0. 0.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 2:
Quaternion (q) = [0. 1. 0. 0.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 3:
Quaternion (q) = [0. 0. 1. 0.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 4:
Quaternion (q) = [0. 0. 0. 1.]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 5:
Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Max difference between Bmat_EP and BmatEP: 0.000000000000e+00
B matrices match.

--------------------------------------------------
Test Case 6:
Quaternion (q) = [0.70710678 0.         0.70710678 0.        ]
Max differen

# 4) BInvmat_EP

In [9]:
def BInvmat_EP(q, convention="scalar_first"):
    """
    Computes the 3x4 B matrix that maps the derivative of the quaternion (Euler parameters) vector to the body angular velocity (omega).

        omega = 2 * [B(Q)]^(-1) * dQ/dt

    Args:
        q (array-like): A 4-element quaternion (Euler parameter) vector [q0, q1, q2, q3].
        convention (str): Specifies the convention for quaternion representation.
                          Options ---> "scalar_first" (default) or "scalar_last"

    Returns:
        np.ndarray: A 3x4 B matrix.
    
    Notes:
        - The quaternion vector q should be in the form [q0, q1, q2, q3], where q0 is the scalar component.
        - This matrix is used to map quaternion rates to body angular velocity.
    """
    # Validate the input quaternion vector
    validate_vec4(q)

    # Convert input to a NumPy array if not already
    q = np.array(q, dtype=float)

    # Extract components of the quaternion
    if convention == "scalar_first":
        q0, q1, q2, q3 = q
    elif convention == "scalar_last":
        q1, q2, q3, q0 = q

    # Construct the BInv matrix using a structured array
    B_inv = np.array([[-q1,  q0,  q3, -q2],
                      [-q2, -q3,  q0,  q1],
                      [-q3,  q2, -q1,  q0]])

    return B_inv
    
BInvmat_EP([0,1,2,1])

array([[-1.,  0.,  1., -2.],
       [-2., -1.,  0.,  1.],
       [-1.,  2., -1.,  0.]])

## 4.1 - Functional testing of BInvmat_EP

In [10]:
from pathlib import Path
import sys
import numpy as np

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the BinvEP function from the AVS lab
from RigidBodyKinematics import BinvEP

# Define test quaternion vectors (Euler Parameters)
test_quaternions = [
    [1, 0, 0, 0],                        # Identity quaternion (no rotation)
    [0, 1, 0, 0],                        # 180-degree rotation about x-axis
    [0, 0, 1, 0],                        # 180-degree rotation about y-axis
    [0, 0, 0, 1],                        # 180-degree rotation about z-axis
    [np.sqrt(0.5), np.sqrt(0.5), 0, 0],  # 90-degree rotation about x-axis
    [np.sqrt(0.5), 0, np.sqrt(0.5), 0],  # 90-degree rotation about y-axis
    [np.sqrt(0.5), 0, 0, np.sqrt(0.5)],  # 90-degree rotation about z-axis
]

# Test each quaternion
for i, q in enumerate(test_quaternions):
    q = np.array(q, dtype=float)
    print(f"Test Case {i + 1}:")
    print(f"Quaternion (q) = {q}")

    # Compute B inverse matrix using the existing BinvEP function
    B_existing = BinvEP(q)

    # Compute B inverse matrix using your BInvmat_EP function
    B_custom = BInvmat_EP(q)

    # Ensure both are NumPy arrays for easy comparison
    B_existing = np.array(B_existing)
    B_custom = np.array(B_custom)

    # Calculate the difference between the two B matrices
    difference = B_existing - B_custom
    max_diff = np.max(np.abs(difference))

    # Print the results
    print(f"Max difference between BInvmat_EP and BinvEP: {max_diff:.12e}")
    if max_diff > 1e-12:
        print("B inverse matrices differ significantly.\n")
    else:
        print("B inverse matrices match.\n")

    print("-" * 50)


Test Case 1:
Quaternion (q) = [1. 0. 0. 0.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 2:
Quaternion (q) = [0. 1. 0. 0.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 3:
Quaternion (q) = [0. 0. 1. 0.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 4:
Quaternion (q) = [0. 0. 0. 1.]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 5:
Quaternion (q) = [0.70710678 0.70710678 0.         0.        ]
Max difference between BInvmat_EP and BinvEP: 0.000000000000e+00
B inverse matrices match.

--------------------------------------------------
Test Case 6:
Quaternion (q) = [0.

# 5) normalize_quat

In [11]:
def normalize_quat(q):
    """
    Normalizes a quaternion to ensure it remains a unit quaternion.

    Args:
        q (array-like): A 4-element quaternion [q0, q1, q2, q3] or [q1, q2, q3, q0]

    Returns:
        np.ndarray: A normalized 4-element quaternion.

    Notes:
        - Ensures the quaternion maintains unit norm, which is critical for rotation representation.
        - Conventions do not matter for this function. The normalization can take place independent of conventions.
    """
    # Validate the input quaternion vector
    validate_vec4(q)

    # Convert input to a NumPy array if not already
    q = np.array(q, dtype=np.float64)

    # Compute the norm of the quaternion
    norm_q = np.linalg.norm(q)

    # Avoid division by zero
    if norm_q == 0:
        raise ValueError("Quaternion norm is zero; cannot normalize.")

    # Normalize the quaternion
    q_normalized = q / norm_q

    return q_normalized

## 5.1 - Functional Testing of normalize_quat

In [12]:
# --- Testing the normalize_quat function ---

# List of test quaternions
test_quaternions = [
    [1, 0, 0, 0],           # Already normalized
    [0, 1, 0, 0],           # Not normalized (but norm is 1; valid unit quaternion)
    [2, 0, 0, 0],           # Scaling factor > 1
    [0, -3, 0, 0],          # Negative value with norm 3
    [1, 2, 3, 4],           # General non-unit quaternion
    [0.1, 0.2, 0.3, 0.4],    # General small values
    [-1, -1, -1, -1],       # Negative quaternion
    [0, 0, 0, 0],           # Zero quaternion (should raise an error)
]

print("Testing normalize_quat function:\n")
for i, q in enumerate(test_quaternions):
    print(f"Test Case {i+1}:")
    print(f"Input Quaternion: {q}")

    # Check for zero quaternion case
    q_np = np.array(q, dtype=float)
    if np.linalg.norm(q_np) == 0:
        print("Expected: ValueError for zero quaternion.")
        try:
            result = normalize_quat(q_np)
        except ValueError as e:
            print("Passed: Caught ValueError:", e)
        else:
            print("FAILED: No error raised for zero quaternion.")
    else:
        # Expected result (q divided by its norm)
        expected = q_np / np.linalg.norm(q_np)
        result = normalize_quat(q_np)
        
        # Calculate norm and difference
        result_norm = np.linalg.norm(result)
        difference = np.linalg.norm(result - expected)
        print(f"Normalized Quaternion: {result}")
        print(f"Norm of result: {result_norm:.12e}")
        print(f"Difference from expected: {difference:.12e}")
        if np.abs(result_norm - 1.0) < 1e-12 and difference < 1e-12:
            print("Passed.")
        else:
            print("FAILED: Normalization error exceeds tolerance.")
    print("-" * 50)

Testing normalize_quat function:

Test Case 1:
Input Quaternion: [1, 0, 0, 0]
Normalized Quaternion: [1. 0. 0. 0.]
Norm of result: 1.000000000000e+00
Difference from expected: 0.000000000000e+00
Passed.
--------------------------------------------------
Test Case 2:
Input Quaternion: [0, 1, 0, 0]
Normalized Quaternion: [0. 1. 0. 0.]
Norm of result: 1.000000000000e+00
Difference from expected: 0.000000000000e+00
Passed.
--------------------------------------------------
Test Case 3:
Input Quaternion: [2, 0, 0, 0]
Normalized Quaternion: [1. 0. 0. 0.]
Norm of result: 1.000000000000e+00
Difference from expected: 0.000000000000e+00
Passed.
--------------------------------------------------
Test Case 4:
Input Quaternion: [0, -3, 0, 0]
Normalized Quaternion: [ 0. -1.  0.  0.]
Norm of result: 1.000000000000e+00
Difference from expected: 0.000000000000e+00
Passed.
--------------------------------------------------
Test Case 5:
Input Quaternion: [1, 2, 3, 4]
Normalized Quaternion: [0.18257419 0.

# 6) quat_mult

**Quaternion Multiplication and Rotation Composition**  

A quaternion is represented as  
$$ q = \begin{bmatrix} w \\ x \\ y \\ z \end{bmatrix} $$  
where **w** is the real part and **(x, y, z)** form the vector part. The function computes the **Hamilton product**, which is used for composing rotations.

**Hamilton Product:**  
Given two quaternions  
$$ q_1 = [w_1, x_1, y_1, z_1], \quad q_2 = [w_2, x_2, y_2, z_2], $$  
the multiplication  
$$ q_{\text{result}} = q_2 * q_1 $$  
is computed as  

$$
q_{\text{result}} =
\begin{bmatrix} 
w_2 & -x_2 & -y_2 & -z_2 \\
x_2 &  w_2 & -z_2 &  y_2 \\
y_2 &  z_2 &  w_2 & -x_2 \\
z_2 & -y_2 &  x_2 &  w_2
\end{bmatrix}
\begin{bmatrix} 
w_1 \\ x_1 \\ y_1 \\ z_1
\end{bmatrix}
$$  

Expanding each component:

$$
\begin{aligned}
w &= w_2 w_1 - x_2 x_1 - y_2 y_1 - z_2 z_1, \\
x &= w_2 x_1 + x_2 w_1 + y_2 z_1 - z_2 y_1, \\
y &= w_2 y_1 - x_2 z_1 + y_2 w_1 + z_2 x_1, \\
z &= w_2 z_1 + x_2 y_1 - y_2 x_1 + z_2 w_1.
\end{aligned}
$$ 

**Intuition:**  
- The multiplication **q₂ * q₁** means that the rotation corresponding to **q₁** is applied first, followed by **q₂**.
- The result is another quaternion **q_result**, which describes the **total equivalent rotation**.

**Why Not Sandwich a Vector?**  
The sandwich product  
$$ v' = q v q^{-1} $$  
is used to rotate a 3D vector **v** using a quaternion. However, this function is not rotating a vector but **combining two rotations** into one. By computing **q₂ * q₁**, we get a new quaternion that represents the **total rotation effect** of first applying **q₁** and then **q₂**. This approach is more efficient than repeatedly applying individual quaternion rotations.

**What Does the Resulting Quaternion Represent?**  
The output **q_result** describes a single quaternion that combines both rotations into one. If this quaternion is later used in a **sandwich product**, it will transform vectors according to both rotations sequentially, without needing to apply **q₁** and **q₂** separately.


In [13]:
def quat_mult(q1, q2, convention="scalar_first"):
    """
    Computes the Hamilton product (quaternion multiplication) with maximum precision 
    using direct computation rather than matrix multiplication. In this function, q1 is 
    treated as the initial quaternion and q2 as the final quaternion. The overall 
    composition is given by: q_result = q2 * q1, which means that the rotation corresponding 
    to q1 is applied first, then q2.
    
    Args:
        q1 (array-like): Initial quaternion [q0, q1, q2, q3] (scalar-first) or [q1, q2, q3, q0] (scalar-last).
        q2 (array-like): Final quaternion   [q0, q1, q2, q3] (scalar-first) or [q1, q2, q3, q0] (scalar-last).
        convention (str): Specifies the quaternion representation.
                          Options: "scalar_first" (default) or "scalar_last".
    
    Returns:
        np.ndarray: The resulting quaternion after multiplication (q2 * q1), maintaining 
                    the input convention.
    """
    # Validate input quaternion vectors
    validate_vec4(q1)
    validate_vec4(q2)

    # Convert to NumPy arrays
    q1 = np.array(q1, dtype=np.float64)
    q2 = np.array(q2, dtype=np.float64)

    # q1 is already in scalar-first: [q0, q1, q2, q3]
    if convention == "scalar_first":
        pass
        
    # Convert from scalar-last [x, y, z, w] to scalar-first [w, x, y, z]
    elif convention == "scalar_last":
        q1 = np.array([q1[3], q1[0], q1[1], q1[2]])
        q2 = np.array([q2[3], q2[0], q2[1], q2[2]])
        
    else:
        raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

    # Normalize quaternions
    q1 = normalize_quat(q1)
    q2 = normalize_quat(q2)

    # Compute Hamilton product with q2 as the left operand and q1 as the right operand.
    # This corresponds to applying the initial rotation (q1) followed by the final rotation (q2).
    w = q2[0]*q1[0] - q2[1]*q1[1] - q2[2]*q1[2] - q2[3]*q1[3]
    x = q2[1]*q1[0] + q2[0]*q1[1] + q2[3]*q1[2] - q2[2]*q1[3]
    y = q2[2]*q1[0] - q2[3]*q1[1] + q2[0]*q1[2] + q2[1]*q1[3]
    z = q2[3]*q1[0] + q2[2]*q1[1] - q2[1]*q1[2] + q2[0]*q1[3]

    q_result = np.array([w, x, y, z], dtype=np.float64)

    # Convert back to scalar-last if needed
    if convention == "scalar_last":
        q_result = np.array([q_result[1], q_result[2], q_result[3], q_result[0]])

    return q_result

## 6.1 - Functional Testing of quat_mult

In [14]:
import numpy as np
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the AVS lab function (reference implementation) for quaternion multiplication.
from RigidBodyKinematics import addEP

# Test quaternion pairs (in scalar-first format: [q0, q1, q2, q3])
# Here, we interpret the first quaternion as B1 and the second as B2.
test_quat_pairs = [
    ([1, 0, 0, 0], [1, 0, 0, 0]),                         # Identity * Identity => Identity
    ([1, 0, 0, 0], [0, 1, 0, 0]),                         # Identity * 180 deg rotation about x-axis
    ([0, 1, 0, 0], [0, 1, 0, 0]),                         # 180 deg rotation about x-axis with itself
    ([np.sqrt(0.5), np.sqrt(0.5), 0, 0],                   # 90 deg rotation about x-axis
     [np.sqrt(0.5), 0, np.sqrt(0.5), 0]),                  # 90 deg rotation about y-axis
    ([np.sqrt(0.5), 0, 0, np.sqrt(0.5)],                   # 90 deg rotation about z-axis
     [np.sqrt(0.5), np.sqrt(0.5), 0, 0]),                  # 90 deg rotation about x-axis
    ([0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]),          # Some random quaternion
]

tolerance = 1e-12

def to_scalar_last(q_sf):
    """ Convert scalar-first quaternion [q0, q1, q2, q3] to scalar-last [q1, q2, q3, q0]. """
    return np.array([q_sf[1], q_sf[2], q_sf[3], q_sf[0]], dtype=np.float64)

def from_scalar_last(q_sl):
    """ Convert scalar-last quaternion [q1, q2, q3, q0] to scalar-first [q0, q1, q2, q3]. """
    return np.array([q_sl[3], q_sl[0], q_sl[1], q_sl[2]], dtype=np.float64)

# === Scalar-First Convention Tests ===
print("\n" + "=" * 80)
print("Testing quat_mult function (Scalar-First Convention)")
print("=" * 80 + "\n")

for i, (q1, q2) in enumerate(test_quat_pairs):
    q1 = np.array(q1, dtype=float)
    q2 = np.array(q2, dtype=float)

    print(f"Test Case {i}:")
    print(f"  q1 = {q1}")
    print(f"  q2 = {q2}")

    # Compute reference result (AVS Lab's addEP)
    q_ref = np.array(addEP(q1.reshape(4, 1), q2.reshape(4, 1))).reshape(-1)

    # Compute custom quaternion multiplication result
    q_custom = quat_mult(q1, q2, convention="scalar_first")

    # Display results
    print(f"  Expected (q_ref):    {q_ref}")
    print(f"  Computed (q_custom): {q_custom}")

    # Compare results and report mismatches
    mismatches = [f"    Index {j}: q_custom={q_custom[j]:.12f}, q_ref={q_ref[j]:.12f}" 
                  for j in range(4) if not np.isclose(q_custom[j], q_ref[j], atol=tolerance)]

    if mismatches:
        print("  ❌ Test FAILED: Mismatch detected!")
        print("\n".join(mismatches))
    else:
        print("  ✅ Test PASSED: Results match within tolerance.")

    print("-" * 80)

# === Scalar-Last Convention Tests ===
print("\n" + "=" * 80)
print("Testing quat_mult function (Scalar-Last Convention)")
print("=" * 80 + "\n")

for i, (q1_sf, q2_sf) in enumerate(test_quat_pairs):
    q1_sl = to_scalar_last(q1_sf)
    q2_sl = to_scalar_last(q2_sf)

    print(f"Test Case {i}:")
    print(f"  q1 (scalar_last) = {q1_sl}")
    print(f"  q2 (scalar_last) = {q2_sl}")

    # Compute reference result (scalar-first → addEP → convert to scalar-last)
    q_ref_sf = np.array(addEP(np.array(q1_sf).reshape(4, 1), np.array(q2_sf).reshape(4, 1))).reshape(-1)
    q_ref_sl = to_scalar_last(q_ref_sf)

    # Compute custom quaternion multiplication result
    q_custom = quat_mult(q1_sl, q2_sl, convention="scalar_last")

    # Display results
    print(f"  Expected (q_ref, scalar_last): {q_ref_sl}")
    print(f"  Computed (q_custom):           {q_custom}")

    # Compare results and report mismatches
    mismatches = [f"    Index {j}: q_custom={q_custom[j]:.12f}, q_ref={q_ref_sl[j]:.12f}" 
                  for j in range(4) if not np.isclose(q_custom[j], q_ref_sl[j], atol=tolerance)]

    if mismatches:
        print("  ❌ Test FAILED: Mismatch detected!")
        print("\n".join(mismatches))
    else:
        print("  ✅ Test PASSED: Results match within tolerance.")

    print("-" * 80)


Testing quat_mult function (Scalar-First Convention)

Test Case 0:
  q1 = [1. 0. 0. 0.]
  q2 = [1. 0. 0. 0.]
  Expected (q_ref):    [1. 0. 0. 0.]
  Computed (q_custom): [1. 0. 0. 0.]
  ✅ Test PASSED: Results match within tolerance.
--------------------------------------------------------------------------------
Test Case 1:
  q1 = [1. 0. 0. 0.]
  q2 = [0. 1. 0. 0.]
  Expected (q_ref):    [0. 1. 0. 0.]
  Computed (q_custom): [0. 1. 0. 0.]
  ✅ Test PASSED: Results match within tolerance.
--------------------------------------------------------------------------------
Test Case 2:
  q1 = [0. 1. 0. 0.]
  q2 = [0. 1. 0. 0.]
  Expected (q_ref):    [-1.  0.  0.  0.]
  Computed (q_custom): [-1.  0.  0.  0.]
  ✅ Test PASSED: Results match within tolerance.
--------------------------------------------------------------------------------
Test Case 3:
  q1 = [0.70710678 0.70710678 0.         0.        ]
  q2 = [0.70710678 0.         0.70710678 0.        ]
  Expected (q_ref):    [0.5 0.5 0.5 0.5]


# 7) quat_inv

In [15]:
def quat_inv(q, convention="scalar_first"):
    """
    Computes the inverse (conjugate) of a quaternion.

    Args:
        q (array-like): Quaternion [q0, q1, q2, q3] (scalar-first) or [q1, q2, q3, q0] (scalar-last).
        convention (str): Specifies the quaternion representation.
                          Options -> "scalar_first" (default) or "scalar_last".

    Returns:
        np.ndarray: The inverse quaternion, maintaining the input convention.

    Notes:
        - If the quaternion is normalized, its inverse is simply the conjugate.
        - If the quaternion is not normalized, this function ensures proper inversion:
          q_inv = q_conjugate / (||q||^2).
        - The returned quaternion will follow the same convention as the input.
    """
    # Validate input quaternion vector
    validate_vec4(q)

    # Convert to NumPy array
    q = np.array(q, dtype=np.float64)

    # Compute the quaternion norm (do not use normalize_quat here)
    norm_q2 = np.dot(q, q)
    if norm_q2 < np.finfo(float).eps:
        raise ValueError("Cannot invert a near-zero quaternion due to precision limitations.")

    # Extract components based on convention and compute conjugate
    if convention == "scalar_first":
        q0, q1, q2, q3 = q
        q_conjugate = np.array([q0, -q1, -q2, -q3], dtype=np.float64)
    elif convention == "scalar_last":
        q1, q2, q3, q0 = q
        q_conjugate = np.array([-q1, -q2, -q3, q0], dtype=np.float64)
    else:
        raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

    # If the quaternion is already normalized, avoid unnecessary division
    if np.abs(norm_q2 - 1.0) < np.finfo(float).eps:
        return q_conjugate  # Already a unit quaternion, no need to divide

    # Compute proper inverse: q_inv = q_conjugate / (norm_q^2)
    q_inv = q_conjugate / norm_q2

    return q_inv

## 7.1 - Functional Testing of quat_inv

In [16]:
import numpy as np
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Assume validate_vec4, normalize_quat, quat_mult, and quat_inv are imported from your library

tolerance = 1e-12

print("Testing quat_inv function (scalar_first convention):\n")
# Define test quaternions in scalar-first format ([q0, q1, q2, q3])
test_quaternions_sf = [
    [1, 0, 0, 0],                  # Identity quaternion
    [0.70710678, 0.70710678, 0, 0],  # 90 degree rotation about x-axis
    [0.5, 0.5, 0.5, 0.5],           # Mixed values
    [0.1, 0.2, 0.3, 0.4],
    [-0.3, 0.4, -0.5, 0.7]
]

for i, q in enumerate(test_quaternions_sf):
    q = np.array(q, dtype=float)
    q = normalize_quat(q)
    q_inv = quat_inv(q, convention="scalar_first")
    
    # For scalar-first, identity quaternion is [1, 0, 0, 0]
    product = quat_mult(q, q_inv, convention="scalar_first")  # q * q_inv
    identity_sf = np.array([1, 0, 0, 0])
    
    mismatches = []
    for j in range(4):
        if not np.isclose(product[j], identity_sf[j], atol=tolerance):
            mismatches.append(f"Index {j}: product={product[j]:.12f}, identity={identity_sf[j]:.12f}")

    print(f"Test {i+1} (scalar_first):")
    print(f"  Original quaternion: {q}")
    print(f"  Inverse:             {q_inv}")
    print(f"  Product:             {product}")
    if mismatches:
        print("Test FAILED: Product does not match identity within tolerance.")
        for mismatch in mismatches:
            print(mismatch)
    else:
        print("Test Passed: Product matches identity exactly within tolerance.")
    print("-" * 50)

print("\n======================================================================================\n")
        
print("Testing quat_inv function (scalar_last convention):\n")

# For scalar_last, convert each test quaternion from scalar-first to scalar-last:
def to_scalar_last(q_sf):
    # q_sf = [q0, q1, q2, q3] becomes [q1, q2, q3, q0]
    return np.array([q_sf[1], q_sf[2], q_sf[3], q_sf[0]])

for i, q_sf in enumerate(test_quaternions_sf):
    q_sf = np.array(q_sf, dtype=float)
    q_sf = normalize_quat(q_sf)
    # Convert to scalar-last representation
    q_sl = to_scalar_last(q_sf)
    
    q_inv = quat_inv(q_sl, convention="scalar_last")
    
    # For scalar-last, the identity quaternion is [0, 0, 0, 1]
    # When multiplying, the correct order is q_inv * q_sl
    product = quat_mult(q_inv, q_sl, convention="scalar_last")  # Compute q_inv * q_sl
    identity_sl = np.array([0, 0, 0, 1])
    
    mismatches = []
    for j in range(4):
        if not np.isclose(product[j], identity_sl[j], atol=tolerance):
            mismatches.append(f"Index {j}: product={product[j]:.12f}, identity={identity_sl[j]:.12f}")

    print(f"Test {i+1} (scalar_last):")
    print(f"  Original quaternion (scalar_last): {q_sl}")
    print(f"  Inverse:             {q_inv}")
    print(f"  Product:             {product}")
    if mismatches:
        print("Test FAILED: Product does not match identity within tolerance.")
        for mismatch in mismatches:
            print(mismatch)
    else:
        print("Test Passed: Product matches identity exactly within tolerance.")
    print("-" * 50)

# --- Testing error handling for invalid input ---
print("Testing error handling for invalid input:")
invalid_quat = [1, 2, 3]  # Not 4 elements
try:
    quat_inv(invalid_quat)
except ValueError as e:
    print("  Passed: Caught error for invalid quaternion input ->", e)
else:
    print("  FAILED: No error raised for invalid quaternion input.")


Testing quat_inv function (scalar_first convention):

Test 1 (scalar_first):
  Original quaternion: [1. 0. 0. 0.]
  Inverse:             [ 1. -0. -0. -0.]
  Product:             [1. 0. 0. 0.]
Test Passed: Product matches identity exactly within tolerance.
--------------------------------------------------
Test 2 (scalar_first):
  Original quaternion: [0.70710678 0.70710678 0.         0.        ]
  Inverse:             [ 0.70710678 -0.70710678 -0.         -0.        ]
  Product:             [1. 0. 0. 0.]
Test Passed: Product matches identity exactly within tolerance.
--------------------------------------------------
Test 3 (scalar_first):
  Original quaternion: [0.5 0.5 0.5 0.5]
  Inverse:             [ 0.5 -0.5 -0.5 -0.5]
  Product:             [1. 0. 0. 0.]
Test Passed: Product matches identity exactly within tolerance.
--------------------------------------------------
Test 4 (scalar_first):
  Original quaternion: [0.18257419 0.36514837 0.54772256 0.73029674]
  Inverse:             

# 8) quat_diff

**_Hamilton Product Definition_**  
For two quaternions:  

$$
q = (q_0, q_1, q_2, q_3), \quad p = (p_0, p_1, p_2, p_3)
$$

their product is given by:

$$
q \otimes p =
\begin{bmatrix}
q_0 p_0 - q_1 p_1 - q_2 p_2 - q_3 p_3 \\
q_0 p_1 + q_1 p_0 + q_2 p_3 - q_3 p_2 \\
q_0 p_2 - q_1 p_3 + q_2 p_0 + q_3 p_1 \\
q_0 p_3 + q_1 p_2 - q_2 p_1 + q_3 p_0
\end{bmatrix}
$$

**_Applying the Hamilton Product to Compute $q_{\text{diff}}$_**  
The relative quaternion is computed as:

$$
q_{\text{diff}} = q_2^{-1} \otimes q_1
$$

where the inverse of a unit quaternion is:

$$
q_2^{-1} = (q_{20}, -q_{21}, -q_{22}, -q_{23})
$$

Substituting this into the multiplication formula:

$$
q_{\text{diff}} =
\begin{bmatrix}
q_{20} q_{10} + q_{21} q_{11} + q_{22} q_{12} + q_{23} q_{13} \\
q_{20} q_{11} - q_{21} q_{10} - q_{22} q_{13} + q_{23} q_{12} \\
q_{20} q_{12} + q_{21} q_{13} - q_{22} q_{10} - q_{23} q_{11} \\
q_{20} q_{13} - q_{21} q_{12} + q_{22} q_{11} - q_{23} q_{10}
\end{bmatrix}
$$

which matches the computation in `quat_diff`:

```python
w =  q2[0] * q1[0] + q2[1] * q1[1] + q2[2] * q1[2] + q2[3] * q1[3]
x = -q2[1] * q1[0] + q2[0] * q1[1] + q2[3] * q1[2] - q2[2] * q1[3]
y = -q2[2] * q1[0] - q2[3] * q1[1] + q2[0] * q1[2] + q2[1] * q1[3]
z = -q2[3] * q1[0] + q2[2] * q1[1] - q2[1] * q1[2] + q2[0] * q1[3]

In [17]:
def quat_diff(q1, q2, convention="scalar_first"):
    """
    Computes the relative quaternion that transforms q2 into q1.

    This is achieved by computing q2_inv * q1, which represents the rotation 
    from q2's frame to q1's frame. The inverse of q2 is computed implicitly 
    (assuming unit quaternions) by negating the vector part of q2.

    Args:
        q1 (array-like): Reference quaternion in either 
                         scalar-first [q0, q1, q2, q3] or 
                         scalar-last [q1, q2, q3, q0] format.
        q2 (array-like): Target quaternion in the same format as q1.
        convention (str): Defines input/output format.
                          Options: "scalar_first" (default) or "scalar_last".

    Returns:
        np.ndarray: The relative quaternion, preserving the input convention.
    """
    # Validate input quaternions
    validate_vec4(q1)
    validate_vec4(q2)

    # Convert to NumPy arrays
    q1 = np.array(q1, dtype=np.float64)
    q2 = np.array(q2, dtype=np.float64)

    # Handle scalar-first vs scalar-last conventions
    if convention == "scalar_first":
        pass
    elif convention == "scalar_last":
        q1 = np.array([q1[3], q1[0], q1[1], q1[2]])
        q2 = np.array([q2[3], q2[0], q2[1], q2[2]])
    else:
        raise ValueError("Convention must be 'scalar_first' or 'scalar_last'.")

    # Normalize quaternions
    q1 = normalize_quat(q1)
    q2 = normalize_quat(q2)

    # Compute the relative quaternion q_diff = q2_inv * q1 without explicit inversion.
    # For unit quaternions, the inverse of q2 is given by [q2[0], -q2[1], -q2[2], -q2[3]].
    # The Hamilton product is computed as follows:
    w =  q2[0] * q1[0] + q2[1] * q1[1] + q2[2] * q1[2] + q2[3] * q1[3]
    x =  q2[0] * q1[1] - q2[1] * q1[0] - q2[2] * q1[3] + q2[3] * q1[2]
    y =  q2[0] * q1[2] + q2[1] * q1[3] - q2[2] * q1[0] - q2[3] * q1[1]
    z =  q2[0] * q1[3] - q2[1] * q1[2] + q2[2] * q1[1] - q2[3] * q1[0]

    q_diff = np.array([w, x, y, z], dtype=np.float64)

    # Convert back to scalar-last if needed
    if convention == "scalar_last":
        q_diff = np.array([q_diff[1], q_diff[2], q_diff[3], q_diff[0]])

    return q_diff

## 8.1 - Functional Testing of quat_diff

In [18]:
import numpy as np
from pathlib import Path
import sys

# Add the directory where RigidBodyKinematics.py is located to sys.path
path_to_rigid_body_kinematics = Path(r"..\..\Codes from AVS Lab")
sys.path.insert(0, str(path_to_rigid_body_kinematics))

# Import the AVS lab function (reference implementation) for relative rotation.
from RigidBodyKinematics import subEP

# List of test quaternion pairs in scalar-first format ([q0, q1, q2, q3])
# Here, we interpret the first quaternion as B1 and the second as B2.
test_quat_pairs = [
    ([1, 0, 0, 0], [1, 0, 0, 0]),                         # Identity * Identity => Identity
    ([1, 0, 0, 0], [0, 1, 0, 0]),                         # Identity to 180 deg rotation about x-axis
    ([0, 1, 0, 0], [0, 1, 0, 0]),                         # 180 deg rotation about x-axis with itself
    
    ([np.sqrt(0.5), np.sqrt(0.5), 0, 0],                   # 90 deg rotation about x-axis
     [np.sqrt(0.5), 0, np.sqrt(0.5), 0]),                  # 90 deg rotation about y-axis
    
    ([np.sqrt(0.5), 0, 0, np.sqrt(0.5)],                   # 90 deg rotation about z-axis
     [np.sqrt(0.5), np.sqrt(0.5), 0, 0]),                  # 90 deg rotation about x-axis

    ([0.4175, 0.3062, 0.7852, 0.3397], [0.2872, 0.6496, 0.3936, 0.5836]),          # Some random quaternion
]

tolerance = 1e-12

def to_scalar_last(q_sf):
    """ Convert scalar-first quaternion [q0, q1, q2, q3] to scalar-last [q1, q2, q3, q0]. """
    return np.array([q_sf[1], q_sf[2], q_sf[3], q_sf[0]], dtype=np.float64)

def from_scalar_last(q_sl):
    """ Convert scalar-last quaternion [q1, q2, q3, q0] to scalar-first [q0, q1, q2, q3]. """
    return np.array([q_sl[3], q_sl[0], q_sl[1], q_sl[2]], dtype=np.float64)

# === Scalar-First Convention Tests ===
print("\n" + "=" * 80)
print("Testing quat_diff function (Scalar-First Convention)")
print("=" * 80 + "\n")

for i, (q1, q2) in enumerate(test_quat_pairs):
    q1 = np.array(q1, dtype=float)
    q2 = np.array(q2, dtype=float)

    print(f"Test Case {i}:")
    print(f"  q1 = {q1}")
    print(f"  q2 = {q2}")

    # Compute reference result (AVS Lab's subEP)
    q_ref = np.array(subEP(q1.reshape(4, 1), q2.reshape(4, 1))).reshape(-1)

    # Compute custom quaternion difference result
    q_custom = quat_diff(q1, q2, convention="scalar_first")

    # Display results
    print(f"  Expected (q_ref):    {q_ref}")
    print(f"  Computed (q_custom): {q_custom}")

    # Compare results and report mismatches
    mismatches = [f"    Index {j}: q_custom={q_custom[j]:.12f}, q_ref={q_ref[j]:.12f}" 
                  for j in range(4) if not np.isclose(q_custom[j], q_ref[j], atol=tolerance)]

    if mismatches:
        print("  ❌ Test FAILED: Mismatch detected!")
        print("\n".join(mismatches))
    else:
        print("  ✅ Test PASSED: Results match within tolerance.")

    print("-" * 80)

# === Scalar-Last Convention Tests ===
print("\n" + "=" * 80)
print("Testing quat_diff function (Scalar-Last Convention)")
print("=" * 80 + "\n")

for i, (q1_sf, q2_sf) in enumerate(test_quat_pairs):
    q1_sl = to_scalar_last(q1_sf)
    q2_sl = to_scalar_last(q2_sf)

    print(f"Test Case {i}:")
    print(f"  q1 (scalar_last) = {q1_sl}")
    print(f"  q2 (scalar_last) = {q2_sl}")

    # Compute reference result (scalar-first → subEP → convert to scalar-last)
    q_ref_sf = np.array(subEP(np.array(q1_sf).reshape(4, 1), np.array(q2_sf).reshape(4, 1))).reshape(-1)
    q_ref_sl = to_scalar_last(q_ref_sf)

    # Compute custom quaternion difference result
    q_custom = quat_diff(q1_sl, q2_sl, convention="scalar_last")

    # Display results
    print(f"  Expected (q_ref, scalar_last): {q_ref_sl}")
    print(f"  Computed (q_custom):           {q_custom}")

    # Compare results and report mismatches
    mismatches = [f"    Index {j}: q_custom={q_custom[j]:.12f}, q_ref={q_ref_sl[j]:.12f}" 
                  for j in range(4) if not np.isclose(q_custom[j], q_ref_sl[j], atol=tolerance)]

    if mismatches:
        print("  ❌ Test FAILED: Mismatch detected!")
        print("\n".join(mismatches))
    else:
        print("  ✅ Test PASSED: Results match within tolerance.")

    print("-" * 80)



Testing quat_diff function (Scalar-First Convention)

Test Case 0:
  q1 = [1. 0. 0. 0.]
  q2 = [1. 0. 0. 0.]
  Expected (q_ref):    [1. 0. 0. 0.]
  Computed (q_custom): [1. 0. 0. 0.]
  ✅ Test PASSED: Results match within tolerance.
--------------------------------------------------------------------------------
Test Case 1:
  q1 = [1. 0. 0. 0.]
  q2 = [0. 1. 0. 0.]
  Expected (q_ref):    [ 0. -1.  0.  0.]
  Computed (q_custom): [ 0. -1.  0.  0.]
  ✅ Test PASSED: Results match within tolerance.
--------------------------------------------------------------------------------
Test Case 2:
  q1 = [0. 1. 0. 0.]
  q2 = [0. 1. 0. 0.]
  Expected (q_ref):    [1. 0. 0. 0.]
  Computed (q_custom): [1. 0. 0. 0.]
  ✅ Test PASSED: Results match within tolerance.
--------------------------------------------------------------------------------
Test Case 3:
  q1 = [0.70710678 0.70710678 0.         0.        ]
  q2 = [0.70710678 0.         0.70710678 0.        ]
  Expected (q_ref):    [ 0.5  0.5 -0.5  0

# 9) quat_kinematics

# 10) integrate_quaternion